<a href="https://colab.research.google.com/github/fricova/lse_twitter_emotions/blob/main/coronatweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install twarc

     |████████████████████████████████| 61kB 3.4MB/s 
  Created wheel for twarc: filename=twarc-2.3.7-cp37-none-any.whl size=49603 sha256=85cb5293ff3009907b28a989977c6f40d90e66d22eb7c5b3fd82dc4121ec6088
  Stored in directory: /root/.cache/pip/wheels/a3/14/96/bfc140625d59cf940bde14b66a1158d08a3062159982822195
  Created wheel for configobj: filename=configobj-5.0.6-cp37-none-any.whl size=34547 sha256=70a8af70eebf786a9e05923ccf761857f8255105a9f39f805457ab4719f82ae5
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built twarc configobj


In [ ]:
# consumer key: V5tRuREEXg18prEZVIpvtbVws
# consumer secret: phhS9oNN5AzoT3RRBmAs9Uch8fAWmVoRqBiXCP8WYu9ojq33gQ
# access token: 1315660253371998208-gBlMomK49nOBVIKlovd0IydHzbh4nA
# access secret: iVAGmxoQ2lFRVcY3Z8gAcZOD5b1IR5sBg5vt5vfpKta1G

In [ ]:
# choice number 2
!twarc configure


Twarc needs to know a few things before it can talk to Twitter on your behalf.


Please enter your Twitter application credentials from apps.twitter.com:

consumer key: V5tRuREEXg18prEZVIpvtbVws
consumer secret: phhS9oNN5AzoT3RRBmAs9Uch8fAWmVoRqBiXCP8WYu9ojq33gQ

How would you like twarc to obtain your user keys?

1) generate access keys by visiting Twitter
2) manually enter your access token and secret

Please enter your choice [1/2] 2
Enter your Access Token: 1315660253371998208-gBlMomK49nOBVIKlovd0IydHzbh4nA
Enter your Access Token Secret: iVAGmxoQ2lFRVcY3Z8gAcZOD5b1IR5sBg5vt5vfpKta1G

The credentials for default have been saved to your configuration file at /root/.twarc

✨ ✨ ✨  Happy twarcing! ✨ ✨ ✨



In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp /content/gdrive/MyDrive/corona_tweets_101-200.zip .

In [ ]:
import zipfile
with zipfile.ZipFile("corona_tweets_101-200.zip") as z:
  z.extractall(".")

In [ ]:
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'black-circle-316613'
bucket_name = 'coronatweetmisha'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
import os
all_files = os.listdir("corona_tweets_101-200")
all_files = list(filter(lambda f: "csv" in f, all_files))
all_files = sorted(all_files)

In [ ]:
all_files

['corona_tweets_101.csv',
 'corona_tweets_102.csv',
 'corona_tweets_103.csv',
 'corona_tweets_104.csv',
 'corona_tweets_105.csv',
 'corona_tweets_106.csv',
 'corona_tweets_107.csv',
 'corona_tweets_108.csv',
 'corona_tweets_109.csv',
 'corona_tweets_110.csv',
 'corona_tweets_111.csv',
 'corona_tweets_112.csv',
 'corona_tweets_113.csv',
 'corona_tweets_114.csv',
 'corona_tweets_115.csv',
 'corona_tweets_116.csv',
 'corona_tweets_117.csv',
 'corona_tweets_118.csv',
 'corona_tweets_119.csv',
 'corona_tweets_120.csv',
 'corona_tweets_121.csv',
 'corona_tweets_122.csv',
 'corona_tweets_123.csv',
 'corona_tweets_124.csv',
 'corona_tweets_125.csv',
 'corona_tweets_126.csv',
 'corona_tweets_127.csv',
 'corona_tweets_128.csv',
 'corona_tweets_129.csv',
 'corona_tweets_130.csv',
 'corona_tweets_131.csv',
 'corona_tweets_132.csv',
 'corona_tweets_133.csv',
 'corona_tweets_134.csv',
 'corona_tweets_135.csv',
 'corona_tweets_136.csv',
 'corona_tweets_137.csv',
 'corona_tweets_138.csv',
 'corona_twe

In [ ]:
from IPython.display import HTML, display
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))
    
def process_ids(file, ids):
  tweets = t.hydrate(iter(ids))
  out = display(progress(0, len(ids)), display_id=True)
  actual_tweets = []
  j = 0
  for tweet in tweets:
    text = tweet['full_text']
    date = tweet['created_at']
    actual_tweets.append({"text": text.replace('\n', '').replace('\r', ''), "date": date})
    j += 1
    out.update(progress(j, len(ids)))
  upload_tweets(file, actual_tweets)

In [ ]:
def upload_tweets(file, actual_tweets):
  df = pd.DataFrame(actual_tweets)
  file_name = file + f".processed.csv"
  df.to_csv(file_name, index=False)
  gcs_service = build('storage', 'v1')
  media = MediaFileUpload(file_name, 
                          mimetype='text/csv',
                          resumable=True)

  request = gcs_service.objects().insert(bucket=bucket_name, 
                                        name=file_name,
                                        media_body=media)

  response = None
  while response is None:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, response = request.next_chunk()

  print('Upload complete')

In [ ]:
import pandas as pd
import twarc
import random
t = twarc.Twarc()
# CHANGE THE NUMBERS HERE
for file in all_files[1:100]:
  data = pd.read_csv("corona_tweets_101-200/"+file)
  ids = data[data.columns[0]].to_list()
  random.shuffle(ids)
  ids = ids[0:100000]
  process_ids(file, ids)

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

Upload complete


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

Upload complete


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

...

Upload complete


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

Upload complete


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    "file_cach

Upload complete
